In [4]:
import numpy as np
import random
import torch
from tool import dataset_DEAL
seed = 3407
WORK = 3      #  3:仇恨目标搜索微调  31:仇恨目标奖励微调  
WORKFILE = 'outputnew_3_CC.json'
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [5]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any! We auto support RoPE Scaling internally!
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
dataset_train,dataset_test = dataset_DEAL(WORKFILE,WORK,seed,test_size=0.1)
def generate_r1_prompt(prompt, target):
    return {"prompt": tokenizer.apply_chat_template(prompt, tokenize=False, continue_final_message=True)+'<|im_end|>', "target": target}
dataset_train = dataset_train.map(lambda x: generate_r1_prompt(x["prompt"], x["target"]))
dataset_test = dataset_test.map(lambda x: generate_r1_prompt(x["prompt"], x["target"]))

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.2. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/3165 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 103,  #101*(8*4) = 3624*0.9
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()
os.makedirs("./SFT", exist_ok=True)
model = model.merge_and_unload()
model.save_pretrained("SFT")  # Local saving
tokenizer.save_pretrained("SFT")

Unsloth: Tokenizing ["prompt"] (num_proc=2):   0%|          | 0/3165 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,165 | Num Epochs = 2 | Total steps = 103
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)


Step,Training Loss
1,2.735700
2,2.827100
3,2.754900
4,2.719400
5,2.584100
6,2.428800
7,2.224100
8,2.175200
9,2.072800
10,1.998100


KeyboardInterrupt: 

In [ ]:
from tool import PRO
PRO['3WorkQwenPrompt_instruction_input'] = """<|im_start|>system
进行'仇恨目标'抽取任务，从句子中抽取作者表达仇恨的群体或个人。仇恨评论通常带有贬义、侮辱性或歧视性，针对特定群体或个人。输出以下段落：俚语分析、语义分析、仇恨目标判断、仇恨目标json输出。<|im_end|>
<|im_start|>user
{}<|im_end|>
<|im_start|>assistant
<think>
"""
prompt =  PRO['3WorkQwenPrompt_instruction_input'] 
FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt.format(
        "那男性和雄性动物的区别呢？使用暴力要有节制。",
    )], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 300)

In [ ]:
# # model.save_pretrained("lora_model")  # Local saving
# # tokenizer.save_pretrained("lora_model")
# # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# if True:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = max_seq_length,
#         dtype = None,
#         load_in_4bit = True,
#     )
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# # alpaca_prompt = You MUST copy from above!

# inputs = tokenizer(
# [
#     prompt.format(
#         "What is a famous tall tower in Paris?", # instruction
#         "", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
# # unsloth/Qwen2.5-3B-Instruct-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-unsloth-bnb-4bit
# # unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-1M-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-1M-unsloth-bnb-4bit
# # @title Show current memory stats
# gpu_stats = torch.cuda.get_device_properties(0)
# start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# print(f"{start_gpu_memory} GB of memory reserved.")
# # @title Show final memory and time stats
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory / max_memory * 100, 3)
# lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
# # print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# # print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")